In [1]:
import mediapipe as mp
import cv2
from math import dist
import numpy as np
from pynput.keyboard import Key, Controller
import pynput as pn
from pynput.mouse import Button
import time
mouse = pn.mouse.Controller()
keyboard = Controller()

In [3]:
def orientation(coordinate_landmark_0, coordinate_landmark_9): #orientation based on 0 and 9 landmarks
    x0 = coordinate_landmark_0[0]
    y0 = coordinate_landmark_0[1]
    
    x9 = coordinate_landmark_9[0]
    y9 = coordinate_landmark_9[1]
    
    if abs(x9 - x0) < 0.05:
        m = 1000000000
    else:
        m = abs((y9 - y0)/(x9 - x0))       #angle created by line joining 0-9 landmarks against horizontal --> tan(x) = m2 - m1 / 1 - m1m2 where m1 = 0 for horizonatl ==> tan(x) = m2 = abs(y9 - y0 / x9 - x0)
        
    if m>=0 and m<=1:
        if x9 > x0:
            return "Right"
        else:
            return "Left"
    if m>1:
        if y9 > y0:       #since, y decreases upwards
            return "Up"
        else:
            return "Down"

In [ ]:
orientation((0, 0), (1, 4))

In [4]:
def finger(x, y, z):   #x, y are landmarks of hand (out of 21) (x represents mid part of finger and y represents tip of the same finger)
    if results.multi_hand_landmarks is not None:
                try:
                    p0x = float(str(results.multi_hand_landmarks[-1].landmark[0]).split('\n')[0].split(" ")[1])   #x coordinate (normalised)
                    p0y = float(str(results.multi_hand_landmarks[-1].landmark[0]).split('\n')[1].split(" ")[1])    #y coordinate (normalised)
                    
                    pmidx = float(str(results.multi_hand_landmarks[-1].landmark[int(x)]).split('\n')[0].split(" ")[1])  #x coordinate of mid point of the finger (landmark x)
                    pmidy = float(str(results.multi_hand_landmarks[-1].landmark[int(x)]).split('\n')[1].split(" ")[1])  #y coordinate "  " ... (landmark x)
                    
                    ptopx = float(str(results.multi_hand_landmarks[-1].landmark[int(y)]).split('\n')[0].split(" ")[1])  #x coordinate of tip of the finger (landmark y)
                    ptopy = float(str(results.multi_hand_landmarks[-1].landmark[int(y)]).split('\n')[1].split(" ")[1])  #y coordinate "  " ... (landmark y)
                    
                    dmid = dist([p0x, p0y], [pmidx, pmidy])
                    dtop = dist([p0x, p0y], [ptopx, ptopy])
                    
                    if z == 0:    #if z=0, will tell which fingers are closed
                        if dmid>dtop:      #if distance of palm(landmark 0) to mid point of finger is greater than distance of palm(landmark 0) to tip of finger
                            if x==7 and y==8:
                                return 1
                            if x==11 and y==12:
                                return 2
                            if x==15 and y==16:
                                return 3
                            if x==19 and y==20:
                                return 4
                    if z == 1:                  #if z=1, returns the coordinates of tip of the finger
                        return (int(1280*ptopx), int(720*ptopy))
                        
                    
                except:
                    pass

In [ ]:
def x_coordinate(landmark):  #landmark out of 21
    return float(str(results.multi_hand_landmarks[-1].landmark[int(landmark)]).split('\n')[0].split(" ")[1])

def y_coordinate(landmark):  #landmark out of 21
    return float(str(results.multi_hand_landmarks[-1].landmark[int(landmark)]).split('\n')[1].split(" ")[1])

def finger(z):   
#is z="finger, it retuens which finger is closed. If z="true coordinate", it returns the true coordinates

    if results.multi_hand_landmarks is not None:
        try:
            p0x = x_coordinate(0) #coordinates of landmark 0
            p0y = y_coordinate(0)
            p7x = x_coordinate(7) #coordinates of tip index
            p7y = y_coordinate(7)
            d07 = dist([p0x, p0y], [p7x, p7y])
          
            p8x = x_coordinate(8) #coordinates of mid index
            p8y = y_coordinate(8)
            d08 = dist([p0x, p0y], [p8x, p8y])
            p11x = x_coordinate(11) #coordinates of tip middlefinger
            p11y = y_coordinate(11)
            d011 = dist([p0x, p0y], [p11x, p11y])
            p12x = x_coordinate(12) #coordinates of mid index
            p12y = y_coordinate(12)                    
            d012 = dist([p0x, p0y], [p12x, p12y])
            p15x = x_coordinate(15) #coordinates of mid index
            p15y = y_coordinate(15)                    
            d015 = dist([p0x, p0y], [p15x, p15y])
            p16x = x_coordinate(16) #coordinates of tip middlefinger
            p16y = y_coordinate(16)
            d016 = dist([p0x, p0y], [p16x, p16y])
            p19x = x_coordinate(19) #coordinates of mid index
            p19y = y_coordinate(19)                    
            d019 = dist([p0x, p0y], [p19x, p19y])
            p20x = x_coordinate(20) #coordinates of mid index
            p20y = y_coordinate(20)                    
            d020 = dist([p0x, p0y], [p20x, p20y])
            close = []
                  
            if z == "finger":    
                if d07>d08:
                    close.append(1)
                if d011>d012:
                    close.append(2)
                if d015>d016:
                    close.append(3)
                if d019>d020:
                    close.append(4)
            return close
        
            if z == "true coordinate":               
                return (int(1280*ptopx), int(720*ptopy))
                        
        except:
            pass

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [7]:
cap = cv2.VideoCapture(0)
mypoints = []
def draw(mypoints):
    for i in range(len(mypoints) - 1):
        cv2.line(image, (mypoints[i][0], mypoints[i][1]), (mypoints[i+1][0], mypoints[i+1][1]), color = (255, 255, 0), thickness = 1)

def lock():
    keyboard.press('q')
    time.sleep(0.2)
    keyboard.release('q')

def move(dir):
    match dir:
        case 'Up':
            keyboard.press('w')
        case 'Down':
            keyboard.press('s')
        case 'Left':
            keyboard.press('a')
        case 'Right':
            keyboard.press('d')

def roll():
    keyboard.press(Key.space)
    time.sleep(0.2)
    keyboard.release(Key.space)

def lightSwing():
    keyboard.press('u')
    time.sleep(0.2)
    keyboard.release('u')

def heavySwing():
    keyboard.press('i')
    time.sleep(0.2)
    keyboard.release('i')

def item():
    keyboard.press('r')
    time.sleep(0.2)
    keyboard.release('r')

def jump():
    keyboard.press('f')
    time.sleep(0.2)
    keyboard.release('f')

def skill():
    keyboard.press(Key.ctrl_l)
    time.sleep(0.2)
    keyboard.release(Key.ctrl_l)

def interact():
    keyboard.press('e')
    time.sleep(0.2)
    keyboard.release('e')

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, img = cap.read()
            image = img.copy()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue
            
            print(success)
            
            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)
            
            #print('Handedness:', results.multi_handedness)

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:                      #returns each landmark with (x, y, z) values. x,y are the normalised coordinates whereas z is the normalised distance from the webcam.
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    
        #     if finger(3, 4, 0) == 5:
        #         cv2.putText(image, "Thumb Closed", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
            
        #     #index finger closed--------------------------------------------------
            
        #     if finger(7, 8, 0) == 1:
        #         cv2.putText(image, "Index Closed", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
            
        #    #index finger closed-------------x------------------x--------------x-------
            
            
        #    #middle finger closed----------------------------------------------------
        
        #     if finger(11, 12, 0) == 2:
        #         cv2.putText(image, "Middle Closed", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
        
        #    #middle finger closed------------x------------------x--------------x--------
            
        #    #ring finger closed--------------------------------------------------------
        
        #     if finger(15, 16, 0) == 3:
        #         cv2.putText(image, "Ring Closed", (500, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
        
        #    #ring finger closed------------x------------------x--------------x---------
            
        #     #little finger closed--------------------------------------------------------
        
        #     if finger(19, 20, 0) == 4:
        #         cv2.putText(image, "Little Closed", (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
        
        #    #little finger closed------------x------------------x--------------x---------
            
        #     #Orientation ---------------------------------------------------------------------
        #     try:
        #         cv2.putText(image, orientation(finger(0, 0, 1), finger(9, 9, 1)),(1000,100), cv2.FONT_HERSHEY_SIMPLEX, 0.9,(0, 255, 0), 2)
        #     except:
        #         pass
            
            
            # WASD
            if finger(7, 8, 0) != 1 and finger(11, 12, 0) != 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) == 4:
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Down":
                    if finger(3,3,1)[0] < finger(17,17,1)[0]:
                        roll()
                    move('Up')
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Up":
                    move('Down')
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Right":
                    if finger(3,3,1)[1] < finger(17,17,1)[1]:
                        roll()
                    move('Right')
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Left":
                    if finger(3,3,1)[1] > finger(17,17,1)[1]:
                        roll()
                    move('Left')
                    
            if finger(7, 8, 0) != 1 and finger(11, 12, 0) == 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) == 4:
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Down":
                    lightSwing()
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Right":
                    if finger(3,3,1)[1] < finger(17,17,1)[1]:
                        item()
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Left":
                    if finger(3,3,1)[1] < finger(17,17,1)[1]:
                        skill() 
            
            if finger(7, 8, 0) != 1 and finger(11, 12, 0) != 2 and finger(15, 16, 0) != 3 and finger(19, 20, 0) == 4:
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Left":
                    if finger(3,3,1)[1] < finger(17,17,1)[1]:
                        interact() 

            if finger(7, 8, 0) != 1 and finger(11, 12, 0) != 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) != 4:
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Down":
                    if finger(3,3,1)[0] < finger(17,17,1)[0]:
                        lock()

            if finger(7, 8, 0) == 1 and finger(11, 12, 0) == 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) != 4:
                if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Down":
                    if finger(3,3,1)[0] < finger(17,17,1)[0]:
                        jump()
            
            if finger(7, 8, 0) == 1 and finger(11, 12, 0) == 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) == 4:
                keyboard.release('w')
                keyboard.release('a')
                keyboard.release('s')
                keyboard.release('d')

            # #Gesture 4 (THUMBS UP FOR OKAY!!)-----------------------------------------------------------------
            # if finger(7, 8, 0) == 1 and finger(11, 12, 0) == 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) == 4:      #if all the fingers are closed
            #     if finger(4, 4, 1)[1] <finger(0, 0, 1)[1]: #y of thumb > y of palm
            #         if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Right":
            #             if finger(3,3,1)[0] < finger(5,5,1)[0]:
            #                 cv2.putText(image, "Okay!!", (500, 200), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 2)
   
            # #Gesture 4 (CALL GESTURE FOR CALL!!)-----------------------------------------------------------------
            # if finger(7, 8, 0) == 1 and finger(11, 12, 0) == 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) != 4:      #if all the fingers are closed
            #     if finger(4, 4, 1)[1] < finger(0, 0, 1)[1]: #y of thumb > y of palm
            #         if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Right":
            #             if finger(4,4,1)[0] < finger(5,5,1)[0]:                        
            #                 cv2.putText(image, "Call!!", (500, 200), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 2)


            # if finger(7, 8, 0) == 1 and finger(11, 12, 0) == 2 and finger(15, 16, 0) == 3 and finger(19, 20, 0) != 4:      #if all the fingers are closed
            #     if orientation(finger(0, 0, 1), finger(9, 9, 1)) == "Up":      
            #         lock()

            #print(results.multi_hand_landmarks)
            
            cv2.imshow('MediaPipe Hands', image)
            if cv2.waitKey(5) & 0xFF == ord("q"):
                break
cap.release()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

: 

In [ ]:
cap = cv2.VideoCapture(0)
mypoints = []
def draw(mypoints):
    for i in range(len(mypoints) - 1):
        cv2.line(image, (mypoints[i][0], mypoints[i][1]), (mypoints[i+1][0], mypoints[i+1][1]), color = (255, 255, 0), thickness = 1)

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, img = cap.read()
            image = img.copy()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)
            
            #print('Handedness:', results.multi_handedness)

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:                      #returns each landmark with (x, y, z) values. x,y are the normalised coordinates whereas z is the normalised distance from the webcam.
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            try:
                output = finger("finger")
                print(output)
                if output == [1,2,3] :
                    keyboard.press(Key.space)
            except:
                pass
            cv2.imshow('MediaPipe Hands', image)
            if cv2.waitKey(5) & 0xFF == ord("q"):
                break
cap.release()

In [ ]:
results.multi_hand_landmarks[-1]

In [ ]:
img.shape

In [ ]:
a = np.array([results.multi_hand_landmarks[-1].landmark[0]])
a

In [ ]:
results.multi_hand_landmarks[-1].landmark[0]

In [ ]:
float(str(results.multi_hand_landmarks[-1].landmark[0]).split('\n')[0].split(" ")[1])

In [ ]:
str(results.multi_hand_landmarks[-1].landmark[0]).split('\n')[2]

In [ ]:
thumb : 0-4 and 0-3
first : 0-8 and 0-6
second : 0-12 and 0-10
third : 0-16 and 0-14
fourth : 0-20 and 0-18

In [ ]:
x = [1,2,3]
if x == [1,2,3]:
    print("true")